# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [612]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
time.sleep(0.2)
import json
from datetime import datetime

In [136]:
KEYWORDS = ['python', 'парсинг', 'Wi-Fi']

In [139]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [151]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
#posts

In [215]:
habr_news = pd.DataFrame()
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, то пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('div', class_='post__text')
    for hub in hubs:
        hub_lower = hub.text.lower()
        # ищем вхождение хотя бы одного желаемого хаба
        if any([desired.lower() in hub_lower for desired in KEYWORDS]):
            title = post.find('a', class_='post__title_link').text
            date = post.find('span', class_="post__time").text
            link = post.find('a', class_='post__title_link').get('href')
            row = {'дата': date, 'заголовок': title, 'ссылка': link}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

,дата,заголовок,ссылка
0,сегодня в 20:09,Умные лампы Hiper,https://habr.com/ru/company/lamptest/blog/554466/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [214]:
def post_text(url_f):
    soup_f = BeautifulSoup(requests.get(url_f).text, 'html.parser')
    return soup_f.find('div', class_="post__text").text

In [217]:
habr_news = pd.DataFrame()
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, то пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('div', class_='post__text')
    for hub in hubs:
        time
        #hub_lower = hub.text.lower()
        # ищем вхождение хотя бы одного желаемого хаба
        title = post.find('a', class_='post__title_link').text
        date = post.find('span', class_="post__time").text
        link = post.find('a', class_='post__title_link').get('href')
        text = post_text(link) 
        row = {'дата': date, 'заголовок': title, 'ссылка': link, 'текст': text}
        if any([desired.lower() in text.lower() for desired in KEYWORDS]):        
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

,дата,заголовок,ссылка,текст
0,сегодня в 20:09,Умные лампы Hiper,https://habr.com/ru/company/lamptest/blog/554466/,"Продолжаю тестировать и изучать «умные» лампы,..."
0,сегодня в 19:21,Операционализация: наука создания показателей ...,https://habr.com/ru/company/skillfactory/blog/...,"Пришло время затронуть психологическую тему, к..."
0,сегодня в 19:19,Facebook отвлекает от работы? Блокируем его пр...,https://habr.com/ru/company/selectel/blog/554256/,\r\nПрокрастинация хорошо знакома каждому из н...
0,сегодня в 17:05,От инвалида до киборга при помощи руки с ИИ,https://habr.com/ru/company/skillfactory/blog/...,Будущее здесь — безо всяких преувеличений. В н...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [350]:
EMAIL = ['xxx@x.ru', 'yyy@y.com', '300195@mail.ru']

In [260]:
#EMAIL = ["300195@mail.ru"]

In [351]:
payload = {'emailAddresses': EMAIL}

In [248]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [258]:
headers = {
'POST': '/v1/web/query/site-breaches/unauthorized-data HTTP/1.1',
'Host': 'identityprotection.avast.com',
'Connection': 'keep-alive',
'Content-Length': '37',
'Accept': 'application/json, text/plain, */*',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Version': '0',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.105 YaBrowser/21.3.2.175 Yowser/2.5 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Content-Type': 'application/json;charset=UTF-8',
'Origin': 'https://www.avast.com',
'Sec-Fetch-Site': 'same-site',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Dest': 'empty',
'Referer': 'https://www.avast.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru,en;q=0.9',
}

In [381]:
response = requests.post(url, headers=headers, json=payload)

In [593]:
results = response.text
results = json.loads(results)
results = pd.DataFrame(results)

In [600]:
def mail_f(breachId_f):
    results_mail = results[results['data'].notnull()].drop(['breaches', 'summary'], axis = 1)
    for st in results_mail['data']:
        for mail, value in st.items():
            for el in value:
                if breachId_f == el['breachId']:
                    return mail

In [620]:
results_data = results[results['breaches'].notnull()].drop(['data', 'summary'], axis = 1)
breaches = pd.DataFrame()
for breach in results_data['breaches']:
    row = {'почта': mail_f(breach['breachId']), 
           'дата утечки': datetime.strptime(breach['publishDate'], '%Y-%m-%dT00:00:00Z'), 
           'источник утечки': breach['site'], 
           'описание утечки': breach['description']}
    breaches = pd.concat([breaches, pd.DataFrame([row])])
breaches

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21,vk.com,"At some time in 2020, the Russian social netwo..."
0,300195@mail.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
0,yyy@y.com,2020-01-03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2021-02-11,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
0,yyy@y.com,2019-06-13,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,yyy@y.com,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`